In [1]:
from web3 import Web3, IPCProvider

In [2]:
w3 = Web3(IPCProvider('/shakespeare/sandipan/.ethereum/geth.ipc'))

In [3]:
w3.eth.getBlock('latest')

AttributeDict({'difficulty': 5791256905298,
 'extraData': HexBytes('0xd783010103844765746887676f312e342e32856c696e7578'),
 'gasLimit': 3141592,
 'gasUsed': 0,
 'hash': HexBytes('0xdc442d2ee4dc26b752bb86a50fed9559d2f1e2b4f0552bd898447733b349ce0e'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'miner': '0x52bc44d5378309EE2abF1539BF71dE1b7d7bE3b5',
 'mixHash': HexBytes('0xd21d1b3f884d071b92c7b6520fba5649d63634a9f6cfe658fc223579c5e790e1'),
 'nonce': HexBytes('0xa5056350ceb68f21'),
 'number': 310848,
 

In [4]:
import pandas as pd

In [7]:
df = pd.read_csv('transaction_till_100000.csv')

In [8]:
df.head()

,From,To,Value,Time,Block
0,ethereum,0,5.000000e+18,0,0
1,ethereum,05a56e2d52c817161883f50c441c3228cfe54d9f,5.000000e+18,1438269988,1
2,ethereum,0193d941b50d91be6567c7ee1c0fe7af498b4137,5.000000e+18,1438270128,10
3,ethereum,e6a7a1d47ff21b6321162aea7c6cb457d5476bca,5.000000e+18,1439799138,100000
4,ethereum,9beb4d346c6309a9ce4f27393c6322c8f03a3be6,5.000000e+18,1438336146,10099


In [17]:
add = df.From.unique()[1]


In [20]:
add = Web3.toChecksumAddress(add)

In [22]:
w3.eth.getBalance(add,100)

140000000000000000000

In [26]:
df_b = df[df['Block']==1000][['From','To']]
    

In [34]:
import numpy as np

In [35]:
np.concatenate(df_b.From.unique(),df_b.To.unique())

TypeError: only integer scalar arrays can be converted to a scalar index

In [38]:
address = np.append(df_b.From.unique(),df_b.To.unique())

In [41]:
blockNum = 100
for i in range(len(address)):
    if address[i]!='ethereum':
        account = Web3.toChecksumAddress(address[i])
        balance = w3.eth.getBalance(account,blockNum)
        print(address[i],balance)

bb7b8287f3f0a933474a79eae42cbca977791171 151250000000000000000


In [43]:
def fillBalanceTable(blockNum):
    df_b = df[df['Block']==blockNum][['From','To']]
    addresses = np.append(df_b.From.unique(),df_b.To.unique())
    for i in range(len(address)):
        if address[i]!='ethereum':
            account = Web3.toChecksumAddress(address[i])
            balance = w3.eth.getBalance(account,blockNum)
            print(address[i],balance)

In [45]:
fillBalanceTable(10000)

bb7b8287f3f0a933474a79eae42cbca977791171 11324843750000000000000
